In [ ]:
# import library
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time


header = {
    "Accept": r"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": r"en-GB,en-US;q=0.9,en;q=0.8",
    "User-Agent": r"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"
}

wines = {}

# url = 'https://www.wine-searcher.com/find/riesling/2021'
# req = requests.get(url, headers=agent)
# content = req.text


# DRIVER_PATH = r'/Users/magnusgreve/Downloads/chromedriver'
# options = Options()
# options.add_argument("--user-data-dir=chrome-data")
# options.headless = False
# options.add_argument("--window-size=1920,1200")
# driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
# time.sleep(5)

def get_wines(year_start, year_end, region):
    for i in range(year_start, year_end):
        response = requests.get(
            f"http://api.scraperapi.com?api_key=c66b14a4fe90203815e58f35e25605a0&url=https://www.wine-searcher.com/find/{region}/{i}")
        if not response.ok:
            print(response.status_code,
                response.text)
            break
        else:
            soup = BeautifulSoup(response.text)
            print('Finding wines')
            cards = soup.findAll(class_='card card-product')
            print('Looping through wines')
            for i in cards:
                name = i.find(class_='card-product__name')
                rating = i.find(class_='badge-rating badge badge-pill')
                try:
                    wines[name.text.replace(
                        "\n", "")] = rating.text.split('/')[0]
                except:
                    wines[name.text.replace("\n", "")] = 0
            print(wines)
            time.sleep(5)
    
    df = pd.DataFrame({'Wines': list(wines.keys()), 'Points': list(wines.values())})
    df['Year'] = df.loc[:,'Wines'].str[:5]
    df['Wines'] = df.loc[:,'Wines'].str[5:]
    return df


df = get_wines(2000, 2022, 'bordeaux')

df.to_csv('Bordueax.csv', index=False)

